In [1]:
from network_simulator.pcc.aurora.schedulers import UDRTrainScheduler
from network_simulator.pcc.aurora.aurora import Aurora
import gym

config_file_path = "config/udr.json"
training_traces = []
train_scheduler = UDRTrainScheduler(
    config_file,
    training_traces,
    percent=args.real_trace_prob,
)

env = gym.make('AuroraEnv-v0', trace_scheduler=train_scheduler)
env.seed(20)
obs = env.reset()
print("The initial observation is {}".format(obs), type(obs))


ImportError: cannot import name 'generate_traces' from 'trace' (/home/mzi/anaconda3/envs/crown/lib/python3.11/trace.py)

In [13]:
from ppo import PPO
from serverless_env import SimEnvironment
from torch import nn as nn
from util import *
import torch

def get_model():
    class ActorNetworkWrapper(nn.Module):
        def __init__(self, input_size=NUM_STATES, hidden_size=HIDDEN_SIZE, 
                output_size=NUM_ACTIONS, base_model=None):
            super(ActorNetworkWrapper, self).__init__()

            self.fc1 = nn.Linear(input_size, hidden_size)
            self.fc1.weight.data, self.fc1.bias.data = \
                base_model.fc1.weight.data, base_model.fc1.bias.data

            self.fc2 = nn.Linear(hidden_size, hidden_size)
            self.fc2.weight.data, self.fc2.bias.data = \
                base_model.fc2.weight.data, base_model.fc2.bias.data

            self.fc3 = nn.Linear(hidden_size, output_size)
            self.fc3.weight.data, self.fc3.bias.data = \
                base_model.fc3.weight.data, base_model.fc3.bias.data

            self.relu = nn.ReLU()

        def forward(self, input_):
            # input_ = torch.FloatTensor(input_)
            output = self.relu(self.fc1(input_))
            output = self.relu(self.fc2(output))
            output = self.fc3(output)

            return output


    env = SimEnvironment("data/readfile_sleep_imageresize_output.csv")
    function_name = env.get_function_name()
    initial_state = env.reset(function_name)
    folder_path = "../model/" + str(function_name)
    agent = PPO(env, function_name, folder_path)
    agent.load_checkpoint("../model/ppo.pth.tar")
    return ActorNetworkWrapper(base_model=agent.actor)

def get_params_argmax(input_size):
    # Take sum of the input vars
    c01 = torch.zeros([1, 1, input_size+1])
    c01[0][0][0] = 1

    c02 = torch.zeros([1, 1, input_size+1])
    c02[0][0][0] = 1
    c02[0][0][-1] = 1

    return c01, c02
    
def get_plain_comparative_firm():
    class MyModel(nn.ModuleList):
        def __init__(self, device=torch.device("cpu")):
            super(MyModel, self).__init__()

            self.input_size = NUM_STATES
            c01, c02 = get_params_argmax(self.input_size)
            self.ft = torch.nn.Flatten()
            
            #################
            # Model
            ################# 
            self.base_model = get_model()
            
            #################
            # Input summation
            #################
            self.input_conv1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=self.input_size+1)
            self.input_conv1.weight = torch.nn.Parameter(c01, requires_grad=True)
            self.input_conv1.bias = torch.nn.Parameter(torch.zeros_like(self.input_conv1.bias, requires_grad=True))
            
            self.input_conv2 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=self.input_size+1)
            self.input_conv2.weight = torch.nn.Parameter(c02, requires_grad=True)
            self.input_conv2.bias = torch.nn.Parameter(torch.zeros_like(self.input_conv2.bias, requires_grad=True))
            
        def forward(self, obs):
            # input processing
            input1 = self.input_conv1(obs)
            input2 = self.input_conv2(obs)
            # the model
            copy1_logits = self.base_model(input1)
            copy2_logits = self.base_model(input2)

            return self.ft(torch.concat((copy1_logits[0], copy2_logits[0]), dim=1))

    return MyModel()
    

In [14]:
model = get_plain_comparative_firm()
x = torch.tensor([[[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]]]) # .to(device="cuda")
print(model(x))

Loading checkpoint...
Checkpoint successfully loaded!
tensor([[-0.4096,  0.3218, -0.4052,  0.2843, -0.0261, -0.0236, -0.4978,  0.3170,
         -0.4891,  0.3435,  0.0267, -0.0081]], grad_fn=<CatBackward0>)


# input range

In [2]:
from ppo import PPO
from serverless_env import SimEnvironment
from torch import nn as nn
from util import *
import torc

env = SimEnvironment("data/readfile_sleep_imageresize_output.csv")
function_name = env.get_function_name()
initial_state = env.reset(function_name)
folder_path = "../model/" + str(function_name)
agent = PPO(env, function_name, folder_path)
agent.load_checkpoint("../model/ppo.pth.tar")


Loading checkpoint...
Checkpoint successfully loaded!


In [4]:
import os

directory = 'data/'
for file in os.listdir(directory):
    print(directory + file)


data/imagerotate_imagecompress_imagecompress_output.csv
data/imageresize_floatoperations_decompress_output.csv
data/writefile_writefile_floatoperations_output.csv
data/sleep_writefile_writefile_output.csv
data/imagecompress_sleep_json2yaml_output.csv
data/compress_matmul_compress_output.csv
data/imagerotate_imageresize_floatoperations_output.csv
data/imagecompress_imageresize_imagerotate_output.csv
data/matmul_readfile_matmul_output.csv
data/compress_json2yaml_compress_output.csv
data/imagerotate_matmul_imageresize_output.csv
data/writefile_sleep_json2yaml_output.csv
data/matmul_writefile_decompress_output.csv
data/readfile_decompress_json2yaml_output.csv
data/compress_compress_floatoperations_output.csv
data/json2yaml_compress_compress_output.csv
data/imageresize_writefile_output.csv
data/sleep_imageresize_imagerotate_output.csv
data/writefile_decompress_readfile_output.csv
data/matmul_compress_imagerotate_output.csv
data/imageresize_imagerotate_decompress_output.csv
data/readfile_jso

In [6]:
import pandas as pd

pd.read_csv("data/compress_imagecompress_writefile_output.csv")

,cpu,memory,duration_std,userDiff,sysDiff,heapTotal,heapUsed,avg_cpu_util,cpu_util_std,slo_preservation,total_cpu_shares,cpu_shares_others,num_containers,arrival_rate,latency,memory_util,memory_util_std,slo_duration
0,1024,1024,111.251007,406440,84994,62001152,50310424,1.083688,0.170170,0.968062,1024,0,1,1,287.505224,0.809760,0.070720,278.322826
1,1024,128,103.914049,392412,53749,41254912,36669184,0.928029,0.146553,0.673212,1024,0,1,1,413.425116,0.887353,0.051544,278.322826
2,1024,1536,65.457717,313265,45048,58466304,42501480,1.424489,0.206987,1.000000,1024,0,1,1,198.411422,0.749802,0.092472,278.322826
3,1024,2048,35.386982,264870,55252,58204160,42878200,1.772631,0.335411,1.000000,1024,0,1,1,129.622706,0.711158,0.095633,278.322826
4,1024,256,100.185774,328919,60483,39211008,33262488,1.091672,0.129765,0.841395,1024,0,1,1,330.787400,0.899257,0.041561,278.322826
5,1024,512,73.346999,378820,30154,50483200,47045464,1.177289,0.122644,0.897280,1024,0,1,1,310.185087,0.877128,0.051665,278.322826
6,128,1024,96.556747,390244,80180,59432960,45585888,1.127244,0.221847,0.949255,128,0,1,1,293.201236,0.776096,0.079610,278.322826
7,128,128,236.147972,303812,45881,38686720,33588344,1.153571,0.346774,0.786116,128,0,1,1,354.048099,0.884630,0.043131,278.322826
8,128,1536,53.889992,284562,69204,57155584,44241936,1.432997,0.235801,1.000000,128,0,1,1,180.209101,0.729126,0.082393,278.322826
9,128,2048,44.046523,282516,42776,57679872,42336600,1.785437,0.279807,1.000000,128,0,1,1,129.833417,0.773912,0.102267,278.322826


data/imagerotate_imagecompress_imagecompress_output.csv
data/imageresize_floatoperations_decompress_output.csv
data/writefile_writefile_floatoperations_output.csv
data/sleep_writefile_writefile_output.csv
data/imagecompress_sleep_json2yaml_output.csv
data/compress_matmul_compress_output.csv
data/imagerotate_imageresize_floatoperations_output.csv
data/imagecompress_imageresize_imagerotate_output.csv
data/matmul_readfile_matmul_output.csv
data/compress_json2yaml_compress_output.csv
data/imagerotate_matmul_imageresize_output.csv
data/writefile_sleep_json2yaml_output.csv
data/matmul_writefile_decompress_output.csv
data/readfile_decompress_json2yaml_output.csv
data/compress_compress_floatoperations_output.csv
data/json2yaml_compress_compress_output.csv
data/imageresize_writefile_output.csv
data/sleep_imageresize_imagerotate_output.csv
data/writefile_decompress_readfile_output.csv
data/matmul_compress_imagerotate_output.csv
data/imageresize_imagerotate_decompress_output.csv
data/readfile_jso

In [15]:
keys = ["avg_cpu_util", "slo_preservation", "total_cpu_shares", "cpu_shares_others", "num_containers", "arrival_rate", "latency", "cpu", "memory"]

for key in keys:
    print(key, env.max_value[key])

avg_cpu_util 0.4864221914477806
slo_preservation 1.0
total_cpu_shares 2048.0
cpu_shares_others 0.0
num_containers 1.0
arrival_rate 1.0
latency 3471.0982984285715
cpu 2048.0
memory 2048.0
